In [4]:
!gdown --id 1--C4P9SSZhYnTITWyYpEzNXZwpOKM6tW

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1--C4P9SSZhYnTITWyYpEzNXZwpOKM6tW
To: /content/dataset.zip
100% 35.0M/35.0M [00:00<00:00, 164MB/s]


In [5]:
!unzip dataset.zip

Archive:  dataset.zip
   creating: dataset/
   creating: dataset/1/
  inflating: dataset/1/1-(1).mp3     
  inflating: dataset/1/10-(1).mp3    
  inflating: dataset/1/11-(1).mp3    
  inflating: dataset/1/12-(1).mp3    
  inflating: dataset/1/13-(1).mp3    
  inflating: dataset/1/14-(1).mp3    
  inflating: dataset/1/15-(1).mp3    
  inflating: dataset/1/16-(1).mp3    
  inflating: dataset/1/17-(1).mp3    
  inflating: dataset/1/18-(1).mp3    
  inflating: dataset/1/19-(1).mp3    
  inflating: dataset/1/2-(1).mp3     
  inflating: dataset/1/20-(1).mp3    
  inflating: dataset/1/21-(1).mp3    
  inflating: dataset/1/22-(1).mp3    
  inflating: dataset/1/23-(1).mp3    
  inflating: dataset/1/24-(1).mp3    
  inflating: dataset/1/25-(1).mp3    
  inflating: dataset/1/26-(1).mp3    
  inflating: dataset/1/27-(1).mp3    
  inflating: dataset/1/28-(1).mp3    
  inflating: dataset/1/29-(1).mp3    
  inflating: dataset/1/3-(1).mp3     
  inflating: dataset/1/30-(1).mp3    
  inflating: dataset

In [1]:
!pip install audiomentations

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**import libraries**

In [84]:
import json
import numpy as np
from glob import glob
import librosa
import librosa.display
import IPython.display as ipd

In [14]:
DATASET_PATH = '/content/dataset/*/*.mp3'

In [67]:
audio_files = glob(DATASET_PATH)

In [16]:
audio_files[0:10]

['/content/dataset/11/15-(11).mp3',
 '/content/dataset/11/21-(11).mp3',
 '/content/dataset/11/13-(11).mp3',
 '/content/dataset/11/36-(11).mp3',
 '/content/dataset/11/1-(11).mp3',
 '/content/dataset/11/20-(11).mp3',
 '/content/dataset/11/17-(11).mp3',
 '/content/dataset/11/40-(11).mp3',
 '/content/dataset/11/12-(11).mp3',
 '/content/dataset/11/29-(11).mp3']

In [17]:
len(audio_files)

480

In [27]:
file_path = "/content/dataset/1/1-(1).mp3"
wav, sr = librosa.load(file_path,sr=None)

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [31]:
wav

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [30]:
wav.shape

(62208,)

In [13]:
ipd.Audio(data=wav,rate=sr)

### **Augmentation**

In [53]:
labels = ['اوراق', 'ارز', 'سکه', 'بانک', 'طلا', 'نفت', 'مشتقات', 'فلزات', 'صندوق سهامی', 'صندوق درآمد ثابت', 'صندوق مختلط', 'صندوق قابل معامله']

In [54]:
len(labels)

12

In [55]:
def add_noise(wav):
  wav_n = wav + 0.009*np.random.normal(0,1,len(wav))
  audio = ipd.Audio(data=wav_n,rate=sr)
  return wav_n, audio

In [56]:
wav_n, audio = add_noise(wav)

In [57]:
wav_n

array([-0.00606822, -0.00825509, -0.00496232, ...,  0.01029987,
        0.01243258,  0.0012666 ])

In [58]:
def shifting_wave_by_times(wav):
  wav_roll = np.roll(wav,int(sr/10))
  audio = ipd.Audio(wav_roll,rate=sr)
  return wav_roll, audio

In [59]:
wav_roll, audio = shifting_wave_by_times(wav)

In [60]:
wav_roll.shape

(62208,)

In [89]:
def stretching_time_by_factor(wav):
  factor = 0.4
  wav_time_stch = librosa.effects.time_stretch(wav,factor)
  audio = ipd.Audio(wav_time_stch,rate=sr)
  return wav_time_stch, audio

In [90]:
wav_time_stch, audio = stretching_time_by_factor(wav)

In [125]:
def pitch_shifting_negative(wav):
  wav_pitch_sf_negative = librosa.effects.pitch_shift(wav,sr,n_steps=-5)
  audio = ipd.Audio(wav_pitch_sf_negative,rate=sr)
  return wav_pitch_sf_negative, audio

In [126]:
wav_pitch_sf_negative, audio = pitch_shifting_negative(wav)

In [127]:
def pitch_shifting_positive(wav):
  wav_pitch_sf_positive = librosa.effects.pitch_shift(wav,sr,n_steps=+5)
  audio = ipd.Audio(wav_pitch_sf_positive,rate=sr)
  return wav_pitch_sf_positive, audio

In [128]:
wav_pitch_sf_positive, audio = pitch_shifting_positive(wav)

In [129]:
DATASET_PATH = '/content/dataset/*/*.mp3'
JSON_PATH = 'data.json'
MAX_LEN = 300

In [130]:
def signal2mfcc(signal, max_len=MAX_LEN):

    mfcc = librosa.feature.mfcc(signal)

    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')
    else:
        mfcc = mfcc[:, :max_len]
    
    return mfcc

In [131]:
def preprocess_audio_files(dataset_path):
  data = {
        "labels": [],
        "MFCCs": []
  }
  counter = 0
  audio_files = glob(dataset_path)
  for i,audio in enumerate(audio_files):
    signal, sample_rate = librosa.load(audio, mono=True, sr=None)
    MFCC = signal2mfcc(signal)
    data["MFCCs"].append(MFCC.tolist())
    data["labels"].append(labels[counter])

    wav_n, audio = add_noise(signal)
    wav_roll, audio = shifting_wave_by_times(signal)
    wav_time_stch, audio = stretching_time_by_factor(signal)
    wav_pitch_sf_negative, audio = pitch_shifting_negative(signal)
    wav_pitch_sf_positive, audio = pitch_shifting_positive(signal)

    wav_n2MFCC = signal2mfcc(wav_n)
    wav_roll2MFCC = signal2mfcc(wav_roll)
    wav_time_stch2MFCC = signal2mfcc(wav_time_stch)
    wav_pitch_sf_negative2MFCC = signal2mfcc(wav_pitch_sf_negative)
    wav_pitch_sf_positive2MFCC = signal2mfcc(wav_pitch_sf_positive)


    data["MFCCs"].append(wav_n2MFCC.tolist())
    data["labels"].append(labels[counter])

    data["MFCCs"].append(wav_roll2MFCC.tolist())
    data["labels"].append(labels[counter])

    data["MFCCs"].append(wav_time_stch2MFCC.tolist())
    data["labels"].append(labels[counter])

    data["MFCCs"].append(wav_pitch_sf_negative2MFCC.tolist())
    data["labels"].append(labels[counter])

    data["MFCCs"].append(wav_pitch_sf_positive2MFCC.tolist())
    data["labels"].append(labels[counter])

    
    if (i+1) % 40 == 0:
      counter += 1
      print(counter)
    
  # save data in json file
  with open(JSON_PATH, "w", encoding='utf-8') as f:
      json.dump(data, f, ensure_ascii=False) 

  return None    

In [132]:
%%time

preprocess_audio_files(DATASET_PATH)

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/cor

1


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/cor

2


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/cor

...

10


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/cor

11


/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")
/usr/local/lib/python3.7/dist-packages/librosa/cor

12
CPU times: user 7min 25s, sys: 2min 4s, total: 9min 29s
Wall time: 8min 32s


In [133]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [136]:
!cp data.json /content/drive/MyDrive/

**load data.json**

In [139]:
!gdown --id 1-322-3NQ84fc2aM2Xg1_LsMf6H8STsZ5

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1-322-3NQ84fc2aM2Xg1_LsMf6H8STsZ5
To: /content/data.json
100% 270M/270M [00:01<00:00, 164MB/s]
